# Import Libraries

In [30]:
import pandas as pd #used for date analysis and manipulation
import numpy as np #used for mathematical functions
from datetime import datetime
import matplotlib.pyplot as plt #used for creating graphs
%matplotlib inline
import seaborn as sns #visulation library based on matplotlib
import folium
from folium import plugins

# Deeper Data Exploration

In [31]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [32]:
#list contents of drive, casesensitive
!ls "/content/drive/My Drive/Colab Notebooks/Abnb Project"

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
AB_NYC_2019.csv  AirBnbPart2.ipynb  LICENSE  README.md


In [26]:
#Change from root to working directory
%cd "/content/drive/My Drive/Colab Notebooks/Abnb Project"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-74c5a85ccb3a>", line 2, in <module>
    get_ipython().magic('cd "/content/drive/My Drive/Colab Notebooks/Abnb Project"')
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-84>", line 2, in cd
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
OSError: [Errno 107] Transport endpoint is not connected


OSError: ignored

In [ ]:
# load data set
df=pd.read_csv('AB_NYC_2019.csv')

In [ ]:
df

In [ ]:
#percentile list
perc=[.25,.50,.75,.95]

#list of data types to include
include=['object', 'Float', 'int']

In [ ]:
#Describe the price column to determine what outliers exist
desc = df["price"].describe(percentiles = perc, include = include)
desc

In [ ]:
#Visualize the top price points and count of listings.
sns.set(rc={'figure.figsize':(10,8)})
viz_bar = price.plot(kind='bar')
viz_bar.set_title('Top Ten Price Points by Listing Count')
viz_bar.set_xlabel('Price')
viz_bar.set_ylabel('Count of Listings')
viz_bar.set_xticklabels(viz_bar.get_xticklabels(),rotation=45)

In [ ]:
#Price Density <ap Before REmoving Outliers
plt.figure(figsize=(14,8))
3sns.scatterplot(x=df['longitude'], y=df['latitude'], hue=df["price"], s=20, palette="rainbow")

In [ ]:
#Determine Quantile Ranges


In [33]:
Q1 = round(df.quantile(0.25))
Q3 = round(df.quantile(0.75))
IQR = round(Q3-Q1)
print(IRQ)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-c4f82f7e3727>", line 1, in <module>
    Q1 = round(df.quantile(0.25))
NameError: name 'df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.

NameError: ignored

In [37]:
df = df[~((df > (Q1-1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-37-3705f464339d>", line 1, in <module>
    df = df[~((df > (Q1-1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
NameError: name 'df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/pyt

NameError: ignored